In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import math

In [2]:
totaldata = []
with open('framingham.csv', mode = 'r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        if 'NA' in lines:
            continue
        else:
            totaldata.append(lines)
traindata = []
for i in range(1,3000):
    traindata.append(list(map(float,np.array(totaldata[i]))))
traindata = np.array(traindata)

nr = np.shape(traindata)[0]
nc = np.shape(traindata)[1]


In [3]:
inputs = np.delete(traindata, nc - 1, 1)
var = np.var(inputs, axis=0)
mean = np.mean(inputs, axis=0)
inputs = (inputs - mean)/var
const = np.ones(nr).reshape(-1,1)
inputs = np.append(inputs, const, axis = 1)
output = traindata[:, -1]
x = inputs.transpose()

In [4]:
def sigm(z):
    return 1/(1+math.exp(-z))

def loss(x,y):
    x = max(x, 1e-15)
    x = min(x, 1-1e-15)
    return float(-y*(math.log(x)) - (1-y)*(math.log(1-x)))

def decision(x):
    if x<= 0.35:
        return 0
    else:
        return 1

lr = 0.1
jo = 1.0
jf = 0.0
jd = jf -jo

cost_array = []

In [5]:
w = np.ones(nc)

while abs(jd) > 0.00001:
    b = np.matmul(w, x)
    sig = np.array(list(map(sigm, b)))
    ls = np.array(list(map(loss, sig, output)))
    jo = np.sum(ls)

    step = np.matmul(sig - output, inputs)

    w = w - lr * step / nr

    jd = jf - jo
    
    jf = jo
    
    cost_array.append(jf)
    
    if jo > jf :
        lr = lr/1.5

print(np.array(cost_array))

[6545.05053977 6467.21869379 6367.88671762 ... 1127.73826282 1127.73403166
 1127.7340338 ]


In [6]:
print(w)

[ 1.07372337e-01  4.41268312e+00 -9.20371555e-02  4.99213655e-02
  1.94241654e+00  4.20487550e-03  3.45456733e-03  8.20677860e-02
  1.06768536e-02  1.47385311e+00  2.35278149e+00  1.12862230e+00
  2.38334273e-01  3.77735525e-02  2.36773546e+00 -1.97696435e+00]


In [7]:
testdata = []
for i in range(3000, len(totaldata)):
    testdata.append(list(map(float,np.array(totaldata[i]))))
testdata = np.array(testdata)

nrt = np.shape(testdata)[0]
nct = np.shape(testdata)[1]

inputs_t = np.delete(testdata, nct - 1, 1)
vart = np.var(inputs_t, axis=0)
meant = np.mean(inputs_t, axis=0)
inputs_t = (inputs_t - meant)/vart
const_t = np.ones(nrt).reshape(-1,1)
inputs_t = np.append(inputs_t, const_t, axis = 1)
output_t = testdata[:, -1]
x_t = inputs_t.transpose()

raw = np.matmul(w, x_t)
raw_predn = np.array(list(map(sigm, raw)))
predn = np.array(list(map(decision, raw_predn)))

In [8]:
diff_output_predn = output_t - predn

fp = np.count_nonzero(diff_output_predn == -1)
fn = np.count_nonzero(diff_output_predn == 1)

actp = np.count_nonzero(output_t == 1)
actn = nrt - actp

print (1- ((fp + fn) / nrt))

print((actp - fn) / actp)

print((actn - fp) / actn)

0.8264840182648402
0.12037037037037036
0.9653916211293261
